In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import re
import random

In [ ]:
 #jika ingin menghapus stopword download package nya terlebih dahulu
 #nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Content Based/KHI/KHI_ITEM_All.csv")
df.head()

,id_product,product_name,description
0,100001,KLASSEN,BT.2022 T.SABIT CHR 64
1,100002,KLASSEN,BT.2022 T.SABIT CHR 96
2,100003,KLASSEN,BT.2022 T.SABIT GP 64
3,100004,KLASSEN,BT.2022 T.SABIT GP 96
4,100005,KLASSEN,BT.2073 T.TZ 18-128


In [ ]:
df.describe()

,id_product
count,221.000000
mean,100117.429864
std,68.045044
min,100001.000000
25%,100057.000000
50%,100118.000000
75%,100177.000000
max,100232.000000


In [ ]:
#merubah type data amount dan id_product menjadi object
df['product_name'] = df['product_name'].apply(lambda x: str(x))
df['id_product'] = df['id_product'].apply(lambda x: str(x))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_product    221 non-null    object
 1   product_name  221 non-null    object
 2   description   221 non-null    object
dtypes: object(3)
memory usage: 5.3+ KB


In [ ]:
def print_description(index):
    example = df[df.index == index][['description', 'id_product', 'product_name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Id:', example[1])
        print('Nama:', example[2])

In [ ]:
print_description(1)

BT.2022 T.SABIT CHR 96
Id: 100002
Nama: KLASSEN 


In [ ]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df['description'].apply(clean_text)

In [ ]:
df.head()

,id_product,product_name,description,desc_clean
0,100001,KLASSEN,BT.2022 T.SABIT CHR 64,bt2022 tsabit chr 64
1,100002,KLASSEN,BT.2022 T.SABIT CHR 96,bt2022 tsabit chr 96
2,100003,KLASSEN,BT.2022 T.SABIT GP 64,bt2022 tsabit gp 64
3,100004,KLASSEN,BT.2022 T.SABIT GP 96,bt2022 tsabit gp 96
4,100005,KLASSEN,BT.2073 T.TZ 18-128,bt2073 ttz 18128


In [ ]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['desc_clean', 'id_product', 'product_name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Id:', example[1])
        print('Nama:', example[2])

In [ ]:
print_description_clean(1)

bt2022 tsabit chr 96
Id: 100002
Nama: KLASSEN 


In [ ]:
#proses TF-IDF dan mencari kemiripan dengan cosine similarity
df.set_index('id_product', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) #proses mencari kemiripan dari matrix td idf dengan cosine
cos_sim #output matrix

array([[1.        , 0.64751628, 0.38290588, ..., 0.        , 0.        ,
        0.        ],
       [0.64751628, 1.        , 0.29288971, ..., 0.        , 0.        ,
        0.        ],
       [0.38290588, 0.29288971, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
# Set index utama di kolom 'id_product'
indices = pd.Series(df.index)
indices[:10]

0    100001
1    100002
2    100003
3    100004
4    100005
5    100006
6    100007
7    100008
8    100009
9    100010
Name: id_product, dtype: object

In [ ]:
pr = pd.DataFrame(cos_sim)

In [ ]:
pr

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220
0,1.000000,0.647516,0.382906,0.292963,0.0,0.000000,0.000000,0.115564,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,0.647516,1.000000,0.292890,0.357961,0.0,0.000000,0.000000,0.000000,0.072110,0.0,0.057776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.061451,0.0,0.0,0.0,0.0,0.061885,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.080219,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,0.382906,0.292890,1.000000,0.643042,0.0,0.000000,0.000000,0.116281,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100580,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.292963,0.357961,0.643042,1.000000,0.0,0.000000,0.000000,0.000000,0.072576,0.0,0.058149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102605,0.061848,0.0,0.0,0.0,0.0,0.062284,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.080737,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.0,0.435645,0.435645,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.203715,0.204348,0.627080,0.198489,0.198489,0.0,0.0,0.039983,0.039983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.586453,0.588276,1.000000,0.571409,0.571409,0.0,0.0
217,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.203715,0.204348,0.198489,0.627080,0.198489,0.0,0.0,0.039983,0.039983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.586453,0.588276,0.571409,1.000000,0.571409,0.0,0.0
218,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.203715,0.204348,0.

In [ ]:
pr.columns = indices

In [ ]:
pr2 = pr.set_index(indices)
pr2.index.names = ['item']

In [ ]:
pr2

id_product,100001,100002,100003,100004,100005,100006,100007,100008,100009,100010,100011,100012,100013,100014,100015,100016,100017,100018,100019,100020,100021,100022,100023,100024,100025,100026,100027,100028,100029,100030,100031,100032,100033,100034,100035,100036,100037,100039,100040,100041,...,100193,100194,100195,100196,100197,100198,100199,100200,100201,100202,100203,100204,100205,100206,100207,100208,100209,100210,100211,100212,100213,100214,100215,100216,100217,100218,100219,100220,100221,100222,100223,100224,100225,100226,100227,100228,100229,100230,100231,100232
item,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,1.000000,0.647516,0.382906,0.292963,0.0,0.000000,0.000000,0.115564,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100002,0.647516,1.000000,0.292890,0.357961,0.0,0.000000,0.000000,0.000000,0.072110,0.0,0.057776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.061451,0.0,0.0,0.0,0.0,0.061885,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.080219,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100003,0.382906,0.292890,1.000000,0.643042,0.0,0.000000,0.000000,0.116281,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100580,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100004,0.292963,0.357961,0.643042,1.000000,0.0,0.000000,0.000000,0.000000,0.072576,0.0,0.058149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102605,0.061848,0.0,0.0,0.0,0.0,0.062284,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.080737,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
100005,0.000000,0.000000,0.000000,0.000000,1.0,0.435645,0.435645,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100228,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.203715,0.204348,0.627080,0.198489,0.198489,0.0,0.0,0.039983,0.039983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.586453,0.588276,1.000000,0.571409,0.571409,0.0,0.0
100229,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.203715,0.204348,0.198489,0.627080,0.198489,0.0,0.0,0.03998

In [ ]:
d = pr2.reset_index()
data_norm = pd.melt(d, id_vars=['item'], value_name='ranking').dropna() #memecah matrix menjadi tabel
print(data_norm.shape)
data_norm.head()

(48841, 3)


,item,id_product,ranking
0,100001,100001,1.000000
1,100002,100001,0.647516
2,100003,100001,0.382906
3,100004,100001,0.292963
4,100005,100001,0.000000


In [ ]:
#df = data_norm
df = data_norm[(data_norm.sum(axis=1) != 0)] #menghilangkan nilai didalam tabel yang bernilai 0

In [ ]:
output = df.sort_values(by=['item','ranking'],ascending=False)

In [ ]:
output

,item,id_product,ranking
48840,100232,100232,1.000000
30276,100232,100146,0.290698
29392,100232,100140,0.192987
48618,100231,100231,1.000000
46408,100231,100221,0.634430
...,...,...,...
221,100001,100002,0.647516
442,100001,100003,0.382906
663,100001,100004,0.292963
1547,100001,100008,0.115564


In [ ]:
output.to_csv('/content/drive/MyDrive/Content Based/KHI/contentbased.csv')

# Bagian testing

In [ ]:
def recommendations(id_product, cos_sim = cos_sim):
    
    recommended_product = []
    
    # Mengambil id produk berdasarkan variabel indicies
    idx = indices[indices == id_product].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:11].index)

    for i in top_10_indexes:
        recommended_product.append(list(df.index)[i])
            
    return recommended_product

In [ ]:
#rekomendasi untuk item (diambil random)
recommendations('117031')

['111927',
 '100521',
 '110682',
 '111908',
 '110895',
 '111945',
 '105756',
 '104616',
 '104562',
 '109476']